In [1]:
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords
import math

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def create_frequency_matrix(sentences):

  frequency_matrix = {}
  stopWords = set(stopwords.words("english"))
  ps = PorterStemmer()

  for sentence in sentences:
    frequency_table = {}
    # word_tokenize - Creates a list of words and punctuation marks
    words = word_tokenize(sentence)
    for word in words:
      word = word.lower()
      word = ps.stem(word)

      # If the word is a stopword
      if word in stopWords:
        continue

      if word in frequency_table:
        frequency_table[word] += 1
      else:
        frequency_table[word] = 1

      '''
      Key Length - Using entire sentence as a key for each sentence might be
      inefficient in terms of memory and readability
      Uniqueness - The first 15 characters are usually enough to distinguish
      between different sentences, especially if the sentences are of significant length.
      However, this could potentially lead to collisions (two different
      sentences having the same first 15 characters), but it's a trade-off
      between key length and uniqueness.
      '''
    frequency_matrix[sentence[:15]] = frequency_table

  return frequency_matrix

In [4]:
def create_term_frequency_matrix(frequency_matrix):

  term_frequency_matrix = {}

  for sentence, frequency_table in frequency_matrix.items():
    term_frequency_table = {}

    count_words_in_sentence = len(frequency_table)
    for word, frequency in frequency_table.items():
      term_frequency_table[word] = frequency / count_words_in_sentence

    term_frequency_matrix[sentence] = term_frequency_table

  return term_frequency_matrix

In [5]:
def count_documents_containing_words(frequency_matrix):

  # Iterate through every document, and update the count if the word is found
  word_per_doc_table = {}
  for sentence, frequency_table in frequency_matrix.items():
    for word, frequency in frequency_table.items():
      if word in word_per_doc_table:
        word_per_doc_table[word] += 1
      else:
        word_per_doc_table[word] = 1

  return word_per_doc_table

In [6]:
def create_idf_matrix(frequency_matrix, word_per_doc_table, total_documents):

  idf_matrix = {}

  for sentence, frequency_table in frequency_matrix.items():
    idf_table = {}

    for word, frequency in frequency_table.items():
      idf_table[word] = math.log10(total_documents / word_per_doc_table[word])

    idf_matrix[sentence] = idf_table

  return idf_matrix

In [7]:
def create_tf_idf_matrix(term_frequency_matrix, idf_matrix):

  tf_idf_matrix = {}
  for (sentence1, frequency_table1), (sentence2, frequency_table2) in zip(term_frequency_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        # Both the tables have the same key
        for (word1, value1), (word2, value2) in zip(frequency_table1.items(), frequency_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sentence1] = tf_idf_table

  return tf_idf_matrix

In [8]:
def score_sentences(tf_idf_matrix):

  sentence_scores = {}

  for sentence, frequency_table in tf_idf_matrix.items():
    total_score_per_sentence = 0
    count_words_in_sentence = len(frequency_table)

    for word, value in frequency_table.items():
      total_score_per_sentence += value

    sentence_scores[sentence] = total_score_per_sentence / count_words_in_sentence

  return sentence_scores

In [9]:
def find_average_score(sentence_scores):

  sum_values = 0
  for key in sentence_scores:
    sum_values += sentence_scores[key]

  average = sum_values / len(sentence_scores)

  return average

In [10]:
def generate_summary(sentences, sentence_scores, threshold):

  sentence_count = 0
  summary = ''

  for sentence in sentences:
    if sentence[:15] in sentence_scores and sentence_scores[sentence[:15]] > threshold:
      summary += " " + sentence
      sentence_count += 1

  return summary

In [11]:
def run_summarization(text):

  # sent_tokens returns a list of sentences from the text
  sentences = sent_tokenize(text)
  # print(sentences)
  # Each sentence is considered as an equivalent to a document
  total_documents = len(sentences)

  frequency_matrix = create_frequency_matrix(sentences)
  # print(frequency_matrix)

  term_frequency_matrix = create_term_frequency_matrix(frequency_matrix)
  # print(term_frequency_matrix)

  # The IDF value for a word is calculated based on the number of documents that contain the word
  count_documents_containing_word = count_documents_containing_words(frequency_matrix)
  # print(count_documents_containing_word)

  idf_matrix = create_idf_matrix(frequency_matrix, count_documents_containing_word, total_documents)
  print(idf_matrix)

  tf_idf_matrix = create_tf_idf_matrix(term_frequency_matrix, idf_matrix)

  sentence_scores = score_sentences(tf_idf_matrix)

  threshold = find_average_score(sentence_scores)

  summary = generate_summary(sentences, sentence_scores, 1.3 * threshold)

  return summary

In [12]:
text = '''
Those Who Are Resilient Stay In The Game Longer
“On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.

I’ve coached mummy and mom clients who gave up after many years toiling away at their respective goal or dream. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep going.”

I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger. Some of you reading this might identify with this message because it resonates with you on a deeper level. For others, at the end of their tether the message might be nothing more than a trivial pep talk. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it.


“Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash
Develop A Powerful Vision Of What You Want
“Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” — Earl Nightingale
I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” That advice has echoed in my mind for decades and became my motivator. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Problems are not stop signs, they are advising you that more work is required to overcome them. Most times, problems help you gain a skill or develop the resources to succeed later. So embrace your challenges and develop the grit to push past them instead of retreat in resignation. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Are you willing to play bigger even if it means repeated failures and setbacks? You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, as long as you’re not regretful later.

If you have not achieved the success you deserve and are considering giving up, will you regret it in a few years or decades from now? Only you can answer that, but you should carve out time to discover your motivation for pursuing your goals. It’s a fact, if you don’t know what you want you’ll get what life hands you and it may not be in your best interest, affirms author Larry Weidel: “Winners know that if you don’t figure out what you want, you’ll get whatever life hands you.” The key is to develop a powerful vision of what you want and hold that image in your mind. Nurture it daily and give it life by taking purposeful action towards it.

Vision + desire + dedication + patience + daily action leads to astonishing success. Are you willing to commit to this way of life or jump ship at the first sign of failure? I’m amused when I read questions written by millennials on Quora who ask how they can become rich and famous or the next Elon Musk. Success is a fickle and long game with highs and lows. Similarly, there are no assurances even if you’re an overnight sensation, to sustain it for long, particularly if you don’t have the mental and emotional means to endure it. This means you must rely on the one true constant in your favour: your personal development. The more you grow, the more you gain in terms of financial resources, status, success — simple. If you leave it to outside conditions to dictate your circumstances, you are rolling the dice on your future.

So become intentional on what you want out of life. Commit to it. Nurture your dreams. Focus on your development and if you want to give up, know what’s involved before you take the plunge. Because I assure you, someone out there right now is working harder than you, reading more books, sleeping less and sacrificing all they have to realise their dreams and it may contest with yours. Don’t leave your dreams to chance.
'''

In [13]:
summarized_text = run_summarization(text)
print(summarized_text)

{'\nThose Who Are ': {'resili': 1.414973347970818, 'stay': 1.414973347970818, 'game': 1.2388820889151366, 'longer': 1.414973347970818, '“': 1.0170333392987803, 'mountain': 1.7160033436347992, 'truth': 1.7160033436347992, 'never': 1.414973347970818, 'climb': 1.7160033436347992, 'vain': 1.7160033436347992, ':': 0.8129133566428556, 'either': 1.7160033436347992, 'reach': 1.7160033436347992, 'point': 1.414973347970818, 'higher': 1.7160033436347992, 'today': 1.7160033436347992, ',': 0.37358066281259295, 'train': 1.7160033436347992, 'power': 1.1139433523068367, 'abl': 1.7160033436347992, 'tomorrow.': 1.7160033436347992, '”': 1.0170333392987803, '—': 1.2388820889151366, 'friedrich': 1.7160033436347992, 'nietzsch': 1.7160033436347992, 'challeng': 1.414973347970818, 'setback': 1.414973347970818, 'meant': 1.7160033436347992, 'defeat': 1.2388820889151366, 'promot': 1.7160033436347992, '.': 0.06279082985945543}, 'However, I real': {'howev': 1.414973347970818, ',': 0.37358066281259295, 'realis': 1.4